In [19]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent  # Assuming notebook is in experiments/notebooks/
sys.path.insert(0, str(project_root))
print(sys.path)
import pandas as pd
import matplotlib.pyplot as plt
from experiments.analysis_utils.data_processing import read_directory
from experiments.analysis_utils.visualization import plot_reaction_speed_distribution, plot_token_distribution

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/miniconda3/envs/radCAD/lib/python312.zip', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12/lib-dynload', '', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12/site-packages']


In [20]:
# Read the data
path_to_runs = Path("../../results/simulations/actors_labelling/")
print(path_to_runs.resolve())
print(path_to_runs.resolve().exists())
proposal_df_full, start_data_df_full, timestep_data_df_full = read_directory(path_to_runs.resolve())

/home/rotsker/PycharmProjects/dg-research-revision/results/simulations/actors_labelling
True


Assume that we sweep over counter_label_percentage. It is how it is called in the start_data_df. Then the cell below should work

In [21]:
from experiments.analysis_utils.metrics import analyze_veto_timing_by_seals

veto_stats = analyze_veto_timing_by_seals(timestep_data_df_full, start_data_df_full, additional_columns=('determining_factor',))
veto_stats['attack_success_rate'] = 100 - veto_stats['veto_rate']
veto_stats['n_successful_attacks'] = veto_stats['attack_success_rate'] / 100 * veto_stats['total_runs']
print("Veto statistics by seal parameters:")
display(veto_stats)

Veto statistics by seal parameters:


veto_rate  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor              
0.01                         0.1                           25                   0.000000   
                                                           37                   0.000000   
                                                           43                   1.079414   
                                                           46                   2.020202   
                                                           62                  91.000000   
                                                           69                  99.100000   
                                                           75                  99.900000   

                                                                               mean_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                      
0.01                         0.1                           25                                NaN   
                                                           37                                NaN   
                                                           43                          37.142857   
                                                           46                          37.500000   
                                                           62                          32.483516   
                                                           69                          30.430878   
                                                           75                          29.096096   

                                                                               median_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                        
0.01                         0.1                           25                                  NaN   
                                                           37                                  NaN   
                                                           43                                 37.0   
                                                           46                                 37.5   
                                                           62                                 32.0   
                                                           69                                 30.0   
                                                           75                                 29.0   

                                                                               hours_mean_time_to_veto_hours  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                                  
0.01                         0.1                           25                                            NaN   
                                                           37                                            NaN   
                                                           43                                     111.428571   
                                                           46                                     112.500000   
                                                           62                                      97.450549   
                                                           69                                      91.292634   
                                                           75                                      87.288288   

                                                                               hours_median_time_to_veto_hours  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                                    
0.01                         0.1                           25                                              NaN   
                                                           37                  

In [26]:
import numpy as np
np.arange(40,79,3)
[40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76]

[40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76]

Now what experiments should we do? I propose binary search approach. Assume we want to find counter_label_percentage=H such that veto_rate is 0%. We know that at H=0 veto_rate is 100%. We also know that at H=100 veto_rate is 0%. You've already tried H=50 and veto_rate was >0%. So we should try H=75. If veto_rate at H=75 is still >0% we should try H=87.5. And so on until we find the value of H where veto_rate is 0%.

Similarly, we can find the value of L such that veto_rate is 100%. We know that at L=0 veto_rate is 100%, and at L=50 veto_rate is <100%. So we should try L=25. If veto_rate at L=25 is still <100% we should try L=12.5. And so on until we find the value of L where veto_rate is 100%.